In [ ]:
from pyspark.sql.functions import col, sum, count, date_format

silver_path = "/mnt/datalake/silver/"
gold_path = "/mnt/datalake/gold/"

fato_vendas_df = spark.read.format("delta").load(f"{silver_path}fato_vendas")

print("Tabela fato da camada Silver carregada.")

print("Calculando faturamento por vendedor...")
agg_vendas_por_vendedor = fato_vendas_df.groupBy("id_vendedor", "nome_vendedor", "equipe") \
    .agg(
        sum("valor_total_item").alias("faturamento_total"),
        count("id_pedido").alias("total_pedidos")
    ) \
    .orderBy(col("faturamento_total").desc())

display(agg_vendas_por_vendedor)

print("Calculando vendas por categoria de produto...")
agg_vendas_por_categoria = fato_vendas_df.groupBy("categoria") \
    .agg(
        sum("valor_total_item").alias("faturamento_total"),
        sum("quantidade").alias("quantidade_total_vendida")
    ) \
    .orderBy(col("faturamento_total").desc())

display(agg_vendas_por_categoria)

print("Calculando faturamento mensal...")
agg_faturamento_mensal = fato_vendas_df.withColumn("ano_mes", date_format(col("data_pedido"), "yyyy-MM")) \
    .groupBy("ano_mes") \
    .agg(
        sum("valor_total_item").alias("faturamento_total")
    ) \
    .orderBy("ano_mes")
    
display(agg_faturamento_mensal)

agg_vendas_por_vendedor.write.format("delta").mode("overwrite").save(f"{gold_path}agg_vendas_por_vendedor")
agg_vendas_por_categoria.write.format("delta").mode("overwrite").save(f"{gold_path}agg_vendas_por_categoria")
agg_faturamento_mensal.write.format("delta").mode("overwrite").save(f"{gold_path}agg_faturamento_mensal")

print("\nCamada Gold criada com sucesso!")